In [3]:
from pathlib import Path
import json
import numpy as np
from sentence_transformers import SentenceTransformer

In [4]:
def load_chunks(jsonl_path):
  chunks = []
  jsonl_path = Path(jsonl_path)
  with open(jsonl_path , "r" ,encoding="utf-8") as f  :
    for line in f :
      chunk = json.loads(line)
      chunks.append(chunk)
  return chunks

In [6]:
cd /content/drive/MyDrive/projects/chu_chat_bot/data

/content/drive/MyDrive/projects/chu_chat_bot/data


In [7]:
chunks = load_chunks("chunks.jsonl")
print(len(chunks))
print(chunks[0]["text"])
print(chunks[0]["source"])
print(chunks[0]["tier"])

1425
PP
Règlement Intérieur
Approuvé par le CA du 05 juin 2014
Mis à jour le 22 mai 2025
RI_UA.pdf
2


In [8]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [9]:
texts = [c["text"] for c in chunks]
texts[0]

'PP\nRèglement Intérieur\nApprouvé par le CA du 05 juin 2014\nMis à jour le 22 mai 2025'

In [10]:
embeddings = model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True
)

embeddings = np.asarray(embeddings, dtype=np.float32)


In [11]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 72.0 MB/s eta 0:00:00


In [12]:
import faiss
embeddings.shape

(1425, 384)

In [13]:
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)


In [ ]:
ls

In [14]:
from pathlib import Path

index_dir = Path("index")
index_dir.mkdir(parents=True, exist_ok=True)

faiss.write_index(index, str(index_dir / "index.faiss"))

with open(index_dir / "chunks_metadata.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)


In [15]:
def search(query, k=5):
    query_embedding = model.encode( #query to a vector
        [query],
        normalize_embeddings=True
    )
    query_embedding = np.asarray(query_embedding, dtype=np.float32)

    scores, indices = index.search(query_embedding, k) # looks for the closest vectors in the meaning to the query

    results = []
    for score, idx in zip(scores[0], indices[0]): # cuz score is 2d [[x,x,x]]
        results.append({
            "score": float(score),
            "chunk": chunks[idx]
        })

    return results


In [19]:
results = search("Comment demander un congé maladie ?")

for r in results:
    print(r["score"])
    print(r["chunk"]["source"], r["chunk"]["page"])
    print(r["chunk"]["text"][:300])
    print("-"*50)


0.6171237230300903
reglement-interieur.pdf 246
Règlement intérieur- CH Saumur- Annexes
Combinaison Un agent placé en congé longue durée est en position d'activité. Il a donc droit aux congés
Congé de Longue annuels.
Durée / Congés Les congés annuels non pris au 31/12 de l’année N sont reportés l’année N+1.
Annuels
Si maladie non imputable au Si 
--------------------------------------------------
0.5928235054016113
reglement-interieur.pdf 241
Règlement intérieur- CH Saumur- Annexes
FICHE N°2.4 : CONGE LONGUE MALADIE
Arrêté du 14 mars 1986 établissant la liste des affections ouvrant droit au congé de longue maladie.
Référence statutaire
Pour bénéficier de ce congé, la maladie doit :
• mettre l’intéressé dans l’impossibilité d’exercer ses 
--------------------------------------------------
0.5813008546829224
RI_UA.pdf 70
Règlement intérieur de l’Université d’Angers
Hors congés bonifiés, nul ne peut être autorisé à prendre à quelque titre que ce soit des congés
(congés annuels ou ARTT) exc